In [ ]:
from kafka import KafkaConsumer
from json import loads
from time import sleep
consumer = KafkaConsumer(
    'topic_test',
    bootstrap_servers=['localhost:9094'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    value_deserializer=lambda x: loads(x.decode('utf-8')),
    sasl_plain_username = 'user',
    sasl_plain_password = 'password'
)
for event in consumer:
    event_data = event.value
    # Do whatever you want
    print(event_data)
    sleep(2)

In [ ]:
from kafka import KafkaConsumer
from json import loads
from time import sleep
consumer = KafkaConsumer(
    'topic_test',
    bootstrap_servers=['localhost:9094'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    value_deserializer=lambda x: loads(x.decode('utf-8')),
    sasl_plain_username = 'user',
    sasl_plain_password = 'password'
)

batch_size = 100  # Adjust this value to set your desired batch size

try:
    while True:
        # Poll for new messages
        batch = consumer.poll(max_records=batch_size, timeout_ms=1000)

        if batch:
            for message in batch.values():
                # Your processing logic for each message
                for i in message:
                    print("Received message: {}".format(i.value))
            sleep(10)
        else:
            # Handle no messages received
            print("No messages found in the batch.")

except KeyboardInterrupt:
    # Close the consumer gracefully on Ctrl+C
    consumer.close()

In [ ]:
from kafka import KafkaConsumer
from json import loads
from time import sleep
from clickhouse_driver import Client
from datetime import datetime

clickhouse_client = Client('localhost', user='default', password='password',)
date_format = '%Y-%m-%d %H:%M:%S.%f'

consumer = KafkaConsumer(
    'topic_test',
    bootstrap_servers=['localhost:9094'],
    auto_offset_reset='latest',
    enable_auto_commit=True,
    auto_commit_interval_ms=1000,
    value_deserializer=lambda x: loads(x.decode('utf-8')),
    sasl_plain_username = 'user',
    sasl_plain_password = 'password',
    max_partition_fetch_bytes=2147483647
)

try:
    while True:
        # Poll for new messages
        batch = consumer.poll(max_records=1000000,timeout_ms=10000)
        # batch = consumer.poll(timeout_ms=1000)
        if batch:
            for message in batch.values():
                # Your processing logic for each message
                data = []
                for i in message:
                    # print("Received message: {}".format(i.value))
                    i.value['createdAt'] = datetime.strptime('2023-06-30 18:30:00.000', date_format)
                    i.value['updatedAt'] = datetime.strptime('2023-06-30 18:30:00.000', date_format)
                    data.append(i.value)
                query = "INSERT INTO `default`.EventLogs (id, user_type, user_id, device_id, event_category, event_type, event_status, `event_info.key`, `event_info.value`, `source`, ip, createdAt, updatedAt, app_version) VALUES"
                clickhouse_client.execute(query,data)
                print(datetime.now(), "Messages ",len(message)," found in the batch.")
            sleep(10)
        else:
            # Handle no messages received
            print(datetime.now(), "No messages found in the batch.")

except KeyboardInterrupt:
    # Close the consumer gracefully on Ctrl+C
    consumer.close()